<a href="https://colab.research.google.com/github/Venckus/toai_firewall_10mk4/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Imports

In [1]:
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [4]:
# google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1.1 Read all files in `data` directory

In [ ]:
!pwd

/content


In [5]:
!ls 'drive/My Drive/Colab Notebooks/AIacademy/class-C/project/data/'

2021-01-001.log  2021-01-012.log  access.log.11  access.log.21	access.log.5
2021-01-002.log  2021-01-013.log  access.log.12  access.log.22	access.log.6
2021-01-003.log  2021-01-014.log  access.log.13  access.log.23	access.log.7
2021-01-004.log  2021-01-015.log  access.log.14  access.log.24	access.log.8
2021-01-005.log  2021-01-016.log  access.log.15  access.log.25	access.log.9
2021-01-006.log  2021-01-017.log  access.log.16  access.log.26	c2021-01.log
2021-01-007.log  2021-01-018.log  access.log.17  access.log.27	c2021-02.log
2021-01-008.log  2021-01-019.log  access.log.18  access.log.28	c2021-03.log
2021-01-009.log  access.log	  access.log.19  access.log.29	c2021-04.log
2021-01-010.log  access.log.1	  access.log.2	 access.log.3	c2021-05.log
2021-01-011.log  access.log.10	  access.log.20  access.log.4


In [ ]:
!tail -10 'drive/My Drive/Colab Notebooks/AIacademy/class-C/project/data/access.log'

103.149.192.199 - - [17/Oct/2020:06:35:02 +0200] "GET / HTTP/1.1" 200 396 "-" "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36"
213.32.122.82 - - [17/Oct/2020:06:46:43 +0200] "GET / HTTP/1.1" 200 13 "-" "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36"
119.159.225.189 - - [17/Oct/2020:07:14:06 +0200] "GET /admin//config.php HTTP/1.1" 404 232 "-" "curl/7.15.5 (x86_64-redhat-linux-gnu) libcurl/7.15.5 OpenSSL/0.9.8b zlib/1.2.3 libidn/0.6.5"
91.241.19.173 - - [17/Oct/2020:07:23:15 +0200] "\x03\x00\x00/*\xE0\x00\x00\x00\x00\x00Cookie: mstshash=Administr" 400 182 "-" "-"
5.188.210.227 - - [17/Oct/2020:08:22:54 +0200] "\x05\x01\x00" 400 182 "-" "-"
5.188.210.227 - - [17/Oct/2020:08:24:12 +0200] "\x04\x01\x00P\x05\xBC\xD2\xE3\x00" 400 182 "-" "-"
5.188.210.227 - - [17/Oct/2020:08:25:25 +0200] "GET http://5.188.210.227/echo.php HTTP/1.1" 404 209 "https://www.google.com/" "Moz

In [6]:
# read all data files
mypath = 'drive/My Drive/Colab Notebooks/AIacademy/class-C/project/data'
data_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
data_files

['access.log.15',
 'access.log.16',
 'access.log.17',
 'access.log.18',
 'access.log.19',
 'access.log.20',
 'access.log.21',
 'access.log.22',
 'access.log.23',
 'access.log.24',
 'access.log.25',
 'access.log.26',
 'access.log.28',
 'access.log.27',
 'access.log.29',
 'access.log.14',
 'access.log.13',
 'access.log.12',
 'access.log.11',
 'access.log.10',
 'access.log.9',
 'access.log.8',
 'access.log.1',
 'access.log',
 'access.log.2',
 'access.log.3',
 'access.log.4',
 'access.log.5',
 'access.log.6',
 'access.log.7',
 '2021-01-015.log',
 '2021-01-014.log',
 '2021-01-013.log',
 '2021-01-012.log',
 '2021-01-011.log',
 '2021-01-010.log',
 '2021-01-009.log',
 '2021-01-008.log',
 '2021-01-007.log',
 '2021-01-006.log',
 '2021-01-005.log',
 '2021-01-004.log',
 '2021-01-003.log',
 '2021-01-002.log',
 '2021-01-001.log',
 '2021-01-019.log',
 '2021-01-018.log',
 'c2021-05.log',
 '2021-01-017.log',
 'c2021-04.log',
 '2021-01-016.log',
 'c2021-03.log',
 'c2021-02.log',
 'c2021-01.log']

## 1.2 Add all files data into one Pandas dataframe

In [7]:
%%time
li = []
for file in data_files:
    tmp = pd.read_csv(f'{mypath}/{file}', sep='\s+', header=None)
    li.append(tmp)
df = pd.concat(li, axis=0, ignore_index=True)

CPU times: user 440 ms, sys: 58.2 ms, total: 498 ms
Wall time: 15.7 s


# 2 Explore and prepare data

In [8]:
df.head()

,0,1,2,3,4,5,6,7,8,9
0,66.240.205.34,-,-,[26/Sep/2020:07:26:07,+0200],Gh0st\xAD\x00\x00\x00\xE0\x00\x00\x00x\x9CKS``...,400,182,-,-
1,176.113.115.214,-,-,[26/Sep/2020:07:27:21,+0200],GET /vendor/phpunit/phpunit/src/Util/PHP/eval-...,404,209,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
2,199.195.254.38,-,-,[26/Sep/2020:08:04:18,+0200],GET ../../proc/ HTTP,400,182,-,-
3,176.113.115.214,-,-,[26/Sep/2020:08:07:45,+0200],POST /vendor/phpunit/phpunit/src/Util/PHP/eval...,404,209,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
4,156.146.36.72,-,-,[26/Sep/2020:08:19:34,+0200],GET /robots.txt HTTP/1.1,404,232,-,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:5...


In [9]:
df.shape

(83763, 10)

In [10]:
df.isna().sum()

0     0
1     0
2     0
3     0
4     0
5    92
6     0
7     0
8     0
9     0
dtype: int64

In [11]:
df[1].describe()

count     83763
unique        1
top           -
freq      83763
Name: 1, dtype: object

## 2.1 Remove useless data

### 2.1.1 Renove columns

In [12]:
rm_colls =[1,2,4,8]
df.drop(df.columns[rm_colls], axis=1, inplace=True)

Give names to columns

In [13]:
columns = {0:'IP',3:'date',5:'full_request',6:'code',7:'bytes',9:'agent'}
df.rename(columns=columns,inplace=True)

remove rows with `full_request` = nan

In [14]:
df = df[df['full_request'].notna()]

remove '[' from date_time column

In [15]:
df['date'] = df['date'].str[1:]

In [130]:
df.head()

,IP,date,full_request,code,bytes,agent
0,66.240.205.34,26/Sep/2020:07:26:07,Gh0st\xAD\x00\x00\x00\xE0\x00\x00\x00x\x9CKS``...,400,182,-
1,176.113.115.214,26/Sep/2020:07:27:21,GET /vendor/phpunit/phpunit/src/Util/PHP/eval-...,404,209,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
2,199.195.254.38,26/Sep/2020:08:04:18,GET ../../proc/ HTTP,400,182,-
3,176.113.115.214,26/Sep/2020:08:07:45,POST /vendor/phpunit/phpunit/src/Util/PHP/eval...,404,209,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
4,156.146.36.72,26/Sep/2020:08:19:34,GET /robots.txt HTTP/1.1,404,232,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:5...


In [131]:
df.shape

(83671, 6)

### 2.1.2 Remove rows

In [138]:
# drop ssl encoded rows
dft = df[~df['full_request'].str.contains(fr"\\x", regex=True)]
dft.shape

(83231, 6)

In [139]:
dft = dft.join(
    dft['full_request'].str.split(' ', 2, expand=True)
    .rename(columns={0:'method', 1:'request', 2:'http_v'}))

In [140]:
dft.request.isnull().sum()

51

In [141]:
# drop rows where request is `NaN`.
dft.dropna(subset=['request'], inplace=True)

In [142]:
dft = dft[dft['request'] != '/']

In [143]:
dft.request.describe()

count                                                 48899
unique                                                 9640
top       /index.php?mact=CmsJobManager,cntnt01,process,...
freq                                                   2366
Name: request, dtype: object

In [144]:
dft.head()

,IP,date,full_request,code,bytes,agent,method,request,http_v
1,176.113.115.214,26/Sep/2020:07:27:21,GET /vendor/phpunit/phpunit/src/Util/PHP/eval-...,404,209,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,GET,/vendor/phpunit/phpunit/src/Util/PHP/eval-stdi...,HTTP/1.1
2,199.195.254.38,26/Sep/2020:08:04:18,GET ../../proc/ HTTP,400,182,-,GET,../../proc/,HTTP
3,176.113.115.214,26/Sep/2020:08:07:45,POST /vendor/phpunit/phpunit/src/Util/PHP/eval...,404,209,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,POST,/vendor/phpunit/phpunit/src/Util/PHP/eval-stdi...,HTTP/1.1
4,156.146.36.72,26/Sep/2020:08:19:34,GET /robots.txt HTTP/1.1,404,232,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:5...,GET,/robots.txt,HTTP/1.1
5,156.146.36.72,26/Sep/2020:08:19:34,GET /xmlrpc.php?rsd HTTP/1.1,404,232,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:5...,GET,/xmlrpc.php?rsd,HTTP/1.1


In [ ]:
# check if there is even line number for 50% split
if dft.shape[0] % 2 == 1:
  dft = dft[:-1]
dft.shape

(83178, 10)

## 2.2 Split `full_request` column into `method`, `request`, `http version`.

## 2.3 Explore requests

In [192]:
dft.request[1000:1200].values

array(['/sql/myadmin/index.php?lang=en',
       '/sql/webadmin/index.php?lang=en', '/sql/sqlweb/index.php?lang=en',
       '/sql/websql/index.php?lang=en', '/sql/webdb/index.php?lang=en',
       '/sql/sqladmin/index.php?lang=en',
       '/sql/sql-admin/index.php?lang=en',
       '/sql/phpmyadmin2/index.php?lang=en',
       '/sql/phpMyAdmin2/index.php?lang=en',
       '/sql/phpMyAdmin/index.php?lang=en',
       '/db/myadmin/index.php?lang=en', '/db/webadmin/index.php?lang=en',
       '/db/dbweb/index.php?lang=en', '/db/websql/index.php?lang=en',
       '/db/webdb/index.php?lang=en', '/db/dbadmin/index.php?lang=en',
       '/db/db-admin/index.php?lang=en',
       '/db/phpmyadmin3/index.php?lang=en',
       '/db/phpMyAdmin3/index.php?lang=en',
       '/db/phpMyAdmin-3/index.php?lang=en',
       '/administrator/phpmyadmin/index.php?lang=en',
       '/administrator/phpMyAdmin/index.php?lang=en',
       '/administrator/db/index.php?lang=en',
       '/administrator/web/index.php?lang=en',
   

Potential malitious requests:
die, md5, phpunit, .env,

# 3 Data labeling

The source of access logs server is where the Python Flask application runs.

## 3.1 Functions used to filter malicious requests using keywords 

In [146]:
def find_bad(tmp_df, phrase):
    'Find rows by keyword phrase using regex'
    new_df = tmp_df[tmp_df.requests.str.contains(phrase, regex=True, na=False)]
    return new_df

In [147]:
def build_phrase(list):
    tmp = ''
    for word in list:
        if tmp == '': tmp = f'{word}'
        else: tmp = f'{tmp}|{word}'
    return tmp

In [148]:
def get_bad_requests(tmp1_df, keywords):
  bad_requests_phrase = build_phrase(keywords)
  bad_requests_rows = tmp1_df[tmp1_df.request.str.contains(bad_requests_phrase, regex=True, na=False)]
  return bad_requests_rows

### 3.1.1 all bad requests

In [25]:
bad_chars = [
  '\.', '\@', '\{', '\}', '\(', '\)', '\+', '\_', '\;',
]

In [149]:
bad_words = [
    'cgi', 'shell', 'die', 'md5', 'phpunit', 'env', 'sh', 
    'ls', 'cd', 'rm', 'rf', 'wget', 'cmod', 'chown',
    'ZINFO', 'fig', 'script', 'php', 'w00t', 'https', 'com',
    '%6e', '%6c', '%6d', '%61', '%63','%64', '%65', '%66', '%67', '%68', '%69',
    '%7a', '%70', '%72', '%73', '%74', '%75', '%76', '%77', '%78', '%79',
]

### TODO nuo cia galima viska det i pipeline

In [27]:
bad_char_requests = get_bad_requests(dft, bad_chars)
bad_char_requests.shape

(38091, 9)

In [ ]:
bad_word_requests = get_bad_requests(dft, bad_words)
bad_word_requests.shape

(16438, 9)

## 3.2 Good requests

In [ ]:
good_char_requests = bad_char_requests.merge(dft, how='outer', indicator=True).loc[lambda x : x['_merge']=='right_only']
good_word_requests = bad_word_requests.merge(dft, how='outer', indicator=True).loc[lambda x : x['_merge']=='right_only']

In [ ]:
good_char_requests.shape

(45139, 10)

In [ ]:
good_word_requests.shape

(66792, 10)

In [ ]:
bad_requests_3 = good_requests.merge(dft, how='outer', indicator=True).loc[lambda x : x['_merge']=='right_only']

# 4 Model

## 4.1 Splitting the date (training / validation)

In [ ]:
dft['is_safe'] = dft.request.isin(good_requests2.request).astype(bool) # should it be bool?

In [ ]:
dft['is_safe'] = dft.request.isin(good_requests.request).astype(bool)

In [114]:
dft[450:470]

,IP,date,full_request,code,bytes,agent,method,request,http_v,is_safe
496,159.89.196.206,23/Sep/2020:04:17:14,GET /core/.env HTTP/1.1,404,152,python-requests/2.24.0,GET,/core/.env,HTTP/1.1,False
497,159.89.196.206,23/Sep/2020:04:17:16,GET /register?%3Cplay%3Ewithme%3C/%3E HTTP/1.1,404,152,python-requests/2.24.0,GET,/register?%3Cplay%3Ewithme%3C/%3E,HTTP/1.1,True
498,159.89.196.206,23/Sep/2020:04:17:18,POST / HTTP/1.1,405,182,python-requests/2.24.0,POST,/,HTTP/1.1,True
499,159.89.196.206,23/Sep/2020:04:17:18,POST / HTTP/1.1,405,182,python-requests/2.24.0,POST,/,HTTP/1.1,True
500,164.52.24.162,23/Sep/2020:04:18:22,GET / HTTP/1.1,200,13,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:7...,GET,/,HTTP/1.1,True
501,176.113.115.214,23/Sep/2020:04:29:27,GET /?XDEBUG_SESSION_START=phpstorm HTTP/1.1,200,396,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,GET,/?XDEBUG_SESSION_START=phpstorm,HTTP/1.1,False
502,176.113.115.214,23/Sep/2020:04:47:31,GET /?a=fetch&content=<php>die(@md5(HelloThink...,200,396,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,GET,/?a=fetch&content=<php>die(@md5(HelloThinkCMF)...,HTTP/1.1,False
504,187.95.123.122,23/Sep/2020:05:22:42,GET / HTTP/1.1,200,612,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,GET,/,HTTP/1.1,True
505,50.192.43.149,23/Sep/2020:05:41:29,POST /boaform/admin/formPing HTTP/1.1,400,182,polaris botnet,POST,/boaform/admin/formPing,HTTP/1.1,False
506,94.102.51.119,23/Sep/2020:05:44:39,GET / HTTP/1.1,200,396,Linux Gnu (cow),GET,/,HTTP/1.1,True


In [ ]:
queries = dft['request'].to_list()

In [ ]:
y = dft['is_safe'].to_list()

In [ ]:
len(y)

83230

In [ ]:
# 50% split
mid = int(len(y) / 2)
trn_y = y[:mid]
val_y = y[mid:]

In [ ]:
len(trn_y)

41615

In [ ]:
trn = queries[:mid]
val = queries[mid:]
len(queries), len(trn), len(val)

(83230, 41615, 41615)

## 4.2 TfidfVectorizer

In [ ]:
vectorizer = TfidfVectorizer(
    min_df=0.0, 
    analyzer="word", 
    sublinear_tf=True, 
    ngram_range=(1,3))
X = vectorizer.fit_transform(queries)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
lgs = LogisticRegression(class_weight={1: 2 * len(good_requests) / len(bad_requests2), 0: 1.0})

In [ ]:
%%time
lgs.fit(X_train, y_train)

CPU times: user 1.07 s, sys: 1.32 s, total: 2.39 s
Wall time: 1.23 s


LogisticRegression(C=1.0, class_weight={0: 1.0, 1: 8.712381938155001},
                   dual=False, fit_intercept=True, intercept_scaling=1,
                   l1_ratio=None, max_iter=100, multi_class='auto', n_jobs=None,
                   penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
                   verbose=0, warm_start=False)

## 4.3 CountVectorizer - trigram NB features

In [ ]:
veczr = CountVectorizer(analyzer='word')

In [ ]:
%%time
trn_term_doc1 = veczr.fit_transform(trn)
val_term_doc1 = veczr.transform(val)

CPU times: user 411 ms, sys: 522 µs, total: 412 ms
Wall time: 413 ms


In [ ]:
trn_term_doc1

<41590x4172 sparse matrix of type '<class 'numpy.int64'>'
	with 148121 stored elements in Compressed Sparse Row format>

In [ ]:
vocab = veczr1.get_feature_names(); vocab[10:25]

['0bef',
 '0em3eimvipqygz_twxgcbnlicriauqhnu7zbnixomaoiztoilzjrocyjmqavd_bwe',
 '10',
 '100',
 '101',
 '102',
 '104',
 '107',
 '11',
 '110',
 '111',
 '1111',
 '112',
 '113',
 '114']

In [ ]:
w0 = set([o.lower() for o in trn[0].split(' ')]); w0

{'/vendor/phpunit/phpunit/src/util/php/eval-stdin.php', 'get', 'http/1.1'}

In [ ]:
m = LogisticRegression(C=1e8)
m.fit(trn_term_doc1, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=100000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

# 5 Evaluation

## 5.1 tfidf vectorizer

In [51]:
def print_score(m, _df, pred, x_test, y_test):
  fpr, tpr, _ = metrics.roc_curve(y_test, (m.predict_proba(x_test)[:, 1]))
  auc = metrics.auc(fpr, tpr)
  print(f"All rows: {len(_df.IP)}, bad request count: {len(bad_requests)}")
  print("Baseline Constant negative: %.6f" % (len(_df.IP) / (len(_df.IP) + len(bad_requests))))
  print("Accuracy: %f" % m.score(x_test, y_test))  #checking the accuracy
  print("Precision: %f" % metrics.precision_score(y_test, pred))
  print("Recall: %f" % metrics.recall_score(y_test, pred))
  print("F1-Score: %f" % metrics.f1_score(y_test, pred))
  print("AUC: %f" % auc)

In [ ]:
print_score(lgs, dft, predicted, X_test, y_test)

All rows: 83230, bad request count: 7729
Baseline Constant negative: 0.915028
Accuracy: 0.993488
Precision: 0.992875
Recall: 1.000000
F1-Score: 0.996425
AUC: 0.999736


## 5.2 CountVectorizer

In [ ]:
preds = m.predict(val_term_doc1)
(preds==val_y).mean()

0.5762923779754748

# 6 Pipeline

## 6.1 Labeling

In [28]:
def set_label(df_l, bad_data):
  bad_req = get_bad_requests(df_l, bad_data)
  good_req = bad_req.merge(dft, how='outer', indicator=True).loc[lambda x : x['_merge']=='right_only']
  df_l['is_safe'] = df_l.request.isin(good_req.request).astype(bool)
  return df_l, bad_req, good_req

In [150]:
dft, bad_requests, good_requests = set_label(dft, bad_words)

## 6.2 Split and prepare

In [30]:
# analyzer 'word' or 'char'
def get_vectorizers(analyzer):
  t_veczr = TfidfVectorizer(ngram_range=(1,3),
                            min_df=0.0, 
                            analyzer=analyzer, 
                            sublinear_tf=True)
  c_veczr = CountVectorizer(ngram_range=(1,3),
                            analyzer=analyzer, 
                            max_features=10000)
  return t_veczr, c_veczr

In [151]:
dft.shape

(48899, 10)

In [152]:
x = dft['request'].to_list()

In [153]:
y = dft['is_safe'].to_list()

In [154]:
int(len(y) * 0.8)

39119

In [33]:
# train, test, validation : 40% 30% 30%
def split_lst(lst):
  split_trn = int(len(lst) * 0.4)
  trn_lst = lst[:split_trn]
  tmp_lst = lst[split_trn:]
  split_val = int(len(tmp_lst) / 2)
  test_lst = tmp_lst[:split_val]
  val_lst = tmp_lst[split_val:]
  return trn_lst, test_lst, val_lst

In [88]:
# tfidf
def split_val(l):
  mid = int(len(y) * 0.8)
  t = l[:mid]
  v = l[mid:]
  return t, v

In [155]:
# parameter tuning
trn_y, test_y, val_y = split_lst(y)

In [156]:
trn_y, test_y = split_val(y)

In [158]:
len(trn_y), len(test_y) #, len(val_y)

(39119, 9780)

In [36]:
# parameter tuning
trn_x, test_x, val_x = split_lst(x)

In [159]:
trn_x, test_x = split_val(x)

In [160]:
len(trn_x), len(test_x) # , len(val_x)

(39119, 9780)

In [161]:
y[0]

False

In [162]:
t_trn_y, t_val_y = split_val(y)

In [163]:
len(t_trn_y), len(t_val_y)

(39119, 9780)

In [164]:
t_trn_x, t_val_x = split_val(x)

In [43]:
t_veczr, c_veczr = get_vectorizers('word')

In [165]:
%time X = t_veczr.fit_transform(x)

CPU times: user 931 ms, sys: 3.23 ms, total: 935 ms
Wall time: 943 ms


In [166]:
X[0]

<1x23898 sparse matrix of type '<class 'numpy.float64'>'
	with 22 stored elements in Compressed Sparse Row format>

In [167]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 6.3 models

### 6.3.1 TfidfVectorizer

In [47]:
tf_m = LogisticRegression(
    # C=0.1,
    class_weight={1: 2 * len(good_requests) / len(bad_requests), 0: 1.0})

In [187]:
final_model = tf_m.fit(X, y)

In [168]:
%time tf_m.fit(X_train, y_train)

CPU times: user 1.28 s, sys: 1.36 s, total: 2.64 s
Wall time: 1.35 s


LogisticRegression(C=1.0, class_weight={0: 1.0, 1: 8.120452609806545},
                   dual=False, fit_intercept=True, intercept_scaling=1,
                   l1_ratio=None, max_iter=100, multi_class='auto', n_jobs=None,
                   penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
                   verbose=0, warm_start=False)

### 6.3.2 CountVectorizer

In [169]:
%%time
trn_term_doc = c_veczr.fit_transform(trn_x)
val_term_doc = c_veczr.transform(test_x)

CPU times: user 891 ms, sys: 125 µs, total: 892 ms
Wall time: 903 ms


In [170]:
y1 = trn_y
x1 = trn_term_doc.sign()
c_test_x = val_term_doc.sign()
p = x1[y1==1].sum(0)+1
q = x1[y1==0].sum(0)+1
r = np.log((p/p.sum())/(q/q.sum()))
b = np.log(len(p)/len(q))

In [171]:
c_m = LogisticRegression(C=1e8)
c_m.fit(x1, y1)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=100000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Naive Biase with multiplication

## 6.4 Evaluation

### 6.4.1 TfidfVectorizer

In [176]:
predicted = tf_m.predict(X_test)

In [177]:
print_score(tf_m, dft, predicted, X_test, y_test)

All rows: 48899, bad request count: 16438
Baseline Constant negative: 0.748412
Accuracy: 0.980573
Precision: 0.971867
Recall: 0.999847
F1-Score: 0.985658
AUC: 0.999814


In [195]:
# single prediction
val_1 = t_veczr.transform(['/cgi-bin/php?%2D%64+%61%6C%6C%6F%77%5F%75%72%6C%5F%69%6E%63%6C%75%64%65%3D%6F%6E+']) #val_x[109]
# tf_m.predict(val_1)
final_model.predict(val_1)

array([False])

In [ ]:
val_x

### 6.4.2 CountVectorizer

In [172]:
c_preds = c_m.predict(val_term_doc)
(c_preds.T==test_y).mean()

0.9974437627811861

In [186]:
pr = c_veczr.transform(['com'])
c_m.predict(pr)

array([ True])

In [174]:
x_nb = x1.multiply(r)
c_m.fit(x_nb, y1)

LogisticRegression(C=100000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [175]:
val_x_nb = c_test_x.multiply(r)
preds2 = c_m.predict(val_x_nb)
(preds2.T==test_y).mean()

0.7014314928425358

# 7 Grid search

## 7.1 Pipeline

In [ ]:
pipeline = Pipeline([
  ('vect', TfidfVectorizer()),
  ('tfidf', TfidfTransformer()),
  ('clf', SGDClassifier()),
])

In [ ]:
parameters = {
  'vect__max_df': (0.0,), #(0.5, 0.75),
  'vect__max_features': (5000, 10000, 50000),
  'vect__analyzer': ('char', 'word'),
  'vect__ngram_range': ((1, 3), (1, 5)),  # unigrams or bigrams
  'tfidf__use_idf': (True, False),
  # 'tfidf__norm': ('l1', 'l2'),
  'clf__max_iter': (20,),
  'clf__alpha': (0.00001, 0.000001),
  'clf__penalty': ('l2', 'elasticnet'),
  # 'clf__max_iter': (10, 50, 80),
}

In [ ]:
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

In [ ]:
grid_search.fit(list(trn_term_doc), trn_y)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed: 18.2min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed: 41.3min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed: 44.4min finished


AttributeError: ignored

In [ ]:
grid_search.best_score_

nan

# export model

In [188]:
import pickle

model_mypath = 'drive/My Drive/Colab Notebooks/AIacademy/class-C/project/model.pkl'
with open(model_mypath, 'wb') as f:
    pickle.dump(final_model, f)